## Lucas-Kanade Optical Flow

Optical flow describes the movement of points in continuous frames. So we just talk about optical flow in vidoes. And we can use it to track the moving objects. <br>
Optical Flow has many applications, like:
- Structure from Motion
- Video Copmpression
- Video Stablization

The theory of Optical Flow is built based on 2 assumptions:
1. The pixel intensities of an object do not change between consective frams.
2. Neighboring pixels have similar motion.

From the 2 above assumptions, we can get the convolution:
$$
I(x, y, t) = I(x+dx, y+dy, t+dt)
$$
After dividing by $dt$, we can get:
$$
f_x u+f_y v +f_t = 0
$$
where:
$$
f_x = \frac{\partial f}{\partial x}; f_y = \frac{\partial f}{\partial y}
$$$$
u = \frac{dx}{dt},v= \frac{dy}{dt}
$$
The $f$ here I think is just $I$, but it is ambiguous in the offical tutorial.

**Lucas-Kanada Optical Flow**<br>
The derivate function has 2 variables but just one equation. So we can't solve. So we has to use the assumptions before.
We assume the points in 3x3 window around the point will have the same motion. So we can just adapt the coordinates of the 9 points and get the $f_x, f_y$, and solve the equation. (The method of calculating $f_x, f_y$ wasn't given, and I think is just use a small window to represent the range that the central point can be in the next frame, just like what meanshift does. We just need to find the statistic minimal distance place.) With a fixed window, we can't track objects when the motion is large between 2 consective frames, so we just use pyramids to address the problem. Because the motion will be smaller in small scale images<br>
Conclude the derivates of 9 points, we can calculate $u, v$ with Cramer's rule:
$$
\begin{bmatrix} u \\ v \end{bmatrix} = 
\begin{bmatrix} 
\sum_i f_{x_i}^2 & \sum_i f_{x_i}f_{y_i} \\
\sum_i f_{x_i}f_{y_i} & \sum_i f_{y_i}^2 \end{bmatrix} 
\begin{bmatrix}
-\sum_i f_{x_i} f_{t_i} \\ - \sum_i f_{y_i} f_{t_i} 
\end{bmatrix}
$$



In [5]:
import numpy as np
import cv2 as cv
import argparse

img_path = '../img/slow_traffic_small.mp4'

cap = cv.VideoCapture(img_path)

# params for Shi-Tomasi corner detection
featurce_params = dict(maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7)

# params for lucas-Kanade optical flow
lk_params = dict(winSize = (15, 15),
                 maxLevel = 2,
                 criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.3))

color = np.random.randint(0, 256, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **featurce_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while 1:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calculate the optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        frame = cv.circle(frame, (a, b), 5, color[i].tolist(), -1)

    img = cv.add(frame, mask)
    cv.imshow('frame', img)
    k=cv.waitKey(30)&0xff
    if k==27:
        break

    # Now update the previous frames and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv.destroyAllWindows()

We can use Dense Optical Flow to computes the optical flow for all the points in the frame. 

In [1]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('../img/vtest.avi')
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while 1:
    ret, frame2 = cap.read()
    if not ret:
        break
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, pyr_scale=0.5, levels=3, winsize=15,
                                       iterations=3, poly_n=5, poly_sigma=1.2, flags=0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])

    hsv[..., 0] = ang * 180 /(2*np.pi)
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)

    frame = np.hstack((frame2, bgr))
    cv.imshow('frame2', frame)

    k = cv.waitKey(30) & 0xff
    if k==27:
        break

    prvs = next

cap.release()
cv.destroyAllWindows()